In [37]:
from src import genetic_algorithm,differential_evolution
from src.repository.mutation import  PolynomialM
from src.repository.cross_over import SimulatedBC
from src.repository.population import RealP
from src.repository.selection import RandomSelection
from src.repository.encoding import RealEncoding
from src.repository.cross_over import DECrossOver
from src.repository.selection import DESelection
from src.repository.mutation import DEMutation
from scipy.stats import ranksums
from Problems import g1,g4,g5,g6
import math
import pandas as pd
import matplotlib.pyplot as plt


In [38]:
def plot_dataframe(df):
    # Extract the "generation" column for the x-axis
    x = df["generation"]

    # Get the list of columns excluding "generation"
    columns_to_plot = [col for col in df.columns if col != "generation"]

    # Plot each column against the "generation" column
    for col in columns_to_plot:
        y = df[col]
        plt.figure()
        plt.plot(x, y)
        plt.title(f"{col} vs. Generation")
        plt.xlabel("Generation")
        plt.ylabel(col)

    # Display the plots
    plt.show()


In [39]:
g1_bound = [[0,1]]*9
g1_bound.extend([[0, 100],[0,100],[0,100], [0, 1]])

g4_bound = [[78,102],[33,45]]
g4_bound.extend([[27,45]]*3)

g5_bound = [[0,1200],[0,1200],[-0.55,0.55],[-0.55,0.55]]
g6_bound = [[13,100],[0,100]]

g1_encoding = RealEncoding(bounds=g1_bound)
g4_encoding = RealEncoding(bounds=g4_bound)
g5_encoding = RealEncoding(bounds=g5_bound)
g6_encoding = RealEncoding(bounds=g6_bound)

In [40]:
beta = 0.1
def penalty(violation):
    return abs(violation * ( math.sin(violation/beta) + violation))

In [41]:
class G1Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 1e11
        self.constraints = [
            {"func": lambda x: 2 * x[0] + 2 * x[1] + x[9] + x[10], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: 2 * x[0] + 2 * x[2] + x[9] + x[11], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: 2 * x[1] + 2 * x[2] + x[10] + x[11], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: -8 * x[0] + x[9], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -8 * x[1] + x[10], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -8 * x[2] + x[11], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[3] - x[4] + x[9], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[5] - x[6] + x[10], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[7] - x[8] + x[11], "lower_bound": None, "upper_bound": 0}
        ]

    def penalty_function(self,violation):
        return penalty(violation)

    


g1_penalty = G1Penalty()

In [42]:
class G4Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 100000
        self.constraints = [
            {
                "func": lambda x: 85.334407 + 0.0056858*x[1]*x[4] + 0.00026*x[0]*x[3] - 0.0022053*x[2]*x[4],
                "lower_bound": 0,  
                "upper_bound": 92  
            },
            {
                "func": lambda x: 80.51249 + 0.0071217*x[1]*x[4] + 0.0029955*x[0]*x[1] + 0.0021813*x[2]**2,
                "lower_bound": 90,  
                "upper_bound": 110  
            },
            {
                "func": lambda x: 9.300961 + 0.0047026*x[2]*x[4] + 0.0012547*x[0]*x[2] + 0.0019085*x[2]*x[3],
                "lower_bound": 20,  
                "upper_bound": 25  
            },
        ]



    def penalty_function(self,violation):
        return penalty(violation)


g4_penalty = G4Penalty()

In [43]:
class G5Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 1e10
        self.tolerance = 10
        self.constraints = [
            {
                "func": lambda x: x[3]-x[2] + 0.55,
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: x[2]-x[3] + 0.55,
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: 1000 * math.sin(-x[2] - 0.25) + 1000 * math.sin(-x[3] - 0.25) + 894.8 - x[0],
                "lower_bound": 0- self.tolerance ,
                "upper_bound": 0 + self.tolerance
            },

            {
                "func": lambda x: 1000 * math.sin(x[2] - 0.25) + 1000 * math.sin(x[2] - x[3] - 0.25) + 894.8 - x[1],
                "lower_bound": 0 - self.tolerance ,
                "upper_bound": 0 + self.tolerance
            },

            {
                "func": lambda x: 1000 * math.sin(x[3] - 0.25) + 1000 * math.sin(x[3] - x[2] - 0.25) + 1294.8,
                "lower_bound": 0 - self.tolerance ,
                "upper_bound": 0 + self.tolerance
            },

        ]



    def penalty_function(self,violation):    
        return penalty(violation)

g5_penalty = G5Penalty()

In [44]:
class G6Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 1000000
        self.constraints = [
            {
                "func": lambda x: (x[0] - 5)**2 + (x[1] - 5)**2 - 100,
                "lower_bound": 0,  
                "upper_bound": None  
            },
            {
                "func": lambda x: -(x[0] - 6)**2 - (x[1] - 5)**2 + 82.81,
                "lower_bound": 0,  
                "upper_bound": None  
            },
        ]


    def penalty_function(self,violation):
        return penalty(violation)

g6_penalty = G6Penalty()

In [45]:
def print_report(report):
    
    for index, row in report.iterrows():
        metric = row['Metric']
        with_penalty = row['GA']
        de = row['Differential Evolution']
        difference = row['Rank-sum']
        print(f"{metric}:\n")
        
        if metric == 'Best Solution':
            print("  - GA:")
            print(f"    - Score: {with_penalty['best_eval']:.6f}")
            print(f"    - Feasible: {with_penalty['feasible']}\n")
            print(f"    - Solution: {with_penalty['best_solution']}\n")
            print(f"    - Violated Constraints: {with_penalty['violated_constraints']}\n")

            
            print("  - DE:")
            print(f"    - Score: {de['best_eval_de']:.6f}")
            print(f"    - Feasible: {de['feasible_de']}\n")
            print(f"    - Solution: {de['best_solution_de']}\n")
            print(f"    - Violated Constraints: {de['violated_constrainsts_de']}\n")
        else:
            print(f"  - GA:{with_penalty}")
            print(f"  - Differentiial Evolution: {de}")
            print(f'  - Wilcoxon Rank-Sum Test:{str(difference)}\n')


def generate_report(df):

    data_ga = df['best_eval']  
    data_de = df['best_eval_de'] 

    statistic, p_value = ranksums(data_ga, data_de)

    alpha = 0.05 

    std_dev_with_penalty = df['best_eval'].std()
    std_de = df['best_eval_de'].std()

    best_solution_with_penalty = df.loc[df['best_eval'].idxmin()]
    best_de = df.loc[df['best_eval_de'].idxmin()]

    non_feasible_with_penalty = len(df[df['feasible'] == False])
    non_feasible_de = len(df[df['feasible_de'] == False])

    difference = False

    if p_value < alpha:
        difference = True
    else:
        difference = False


    report_data = {
        'Metric': ['Standard Deviation', 'Best Solution', 'Non-Feasible Solutions'],
        'GA': [std_dev_with_penalty, best_solution_with_penalty, non_feasible_with_penalty],
        'Differential Evolution': [std_de, best_de, non_feasible_de],
        'Rank-sum':difference
    }
    report_df = pd.DataFrame(report_data)

    return report_df


    


In [46]:
final_report = {}

problems = [('g1',g1,g1_encoding,g1_penalty),('g4',g4,g4_encoding,g4_penalty),('g5',g5,g5_encoding,g5_penalty),('g6',g6,g6_encoding,g6_penalty)]
#problems = [('g1',g1,g1_encoding,g1_penalty)]

for problem,_,_,_ in problems:
    final_report[problem] = None

for problem_name,problem,encoding_g,penalty_g in problems:
    print(problem_name)
    best_evaluations = []
    best_evaluations_de = []

    feasible = []
    feasible_de = []


    best_solution = []
    best_solution_de = []

    violated_constainsts = []
    violated_constainsts_de = []

    mutation = PolynomialM(r_mut=0.8)
    cross_over = SimulatedBC()
    population = RealP()
    selection = RandomSelection()
    config = genetic_algorithm.GAConfig(150,1000)
    config_de = differential_evolution.DEConfig(150,1000)
    mutation_de = DEMutation(r_mut=0.8)
    cross_over_de = DECrossOver()
    selection_de = DESelection()


    GA = genetic_algorithm.GeneticAlgorithm(cross_over=cross_over,mutation=mutation,population=population,encoding=encoding_g,selection=selection,config=config,penalty=penalty_g)
    DE = differential_evolution.DifferentialEvolution(cross_over=cross_over_de,mutation=mutation_de,population=population,encoding=encoding_g,selection=selection_de,config=config_de,penalty=penalty_g)


    for i in range(30):
        best_de,best_eval_de = DE.run(objective=problem,verbose=False)
        best,best_eval = GA.run(problem,verbose=False)
        
        best_evaluations.append(problem(best))
        best_evaluations_de.append(problem(best_de))

        
        is_feadible_with,violated_c = penalty_g.is_feasible(best)
        is_feasible_de,violated_c_de = penalty_g.is_feasible(best_de)

        feasible.append(is_feadible_with)
        feasible_de.append(is_feasible_de)


        violated_constainsts.append(violated_c)
        violated_constainsts_de.append(violated_c_de)
        
        best_solution.append(best)
        best_solution_de.append(best_de)

    df = {'best_eval':best_evaluations,
          'feasible':feasible,
          'best_eval_de':best_evaluations_de,
          'best_solution':best_solution,
          'best_solution_de':best_solution_de,
          'violated_constraints':violated_constainsts,
          'feasible_de':feasible_de,
          'violated_constrainsts_de':violated_constainsts_de
          }

    
    final_report[problem_name] = pd.DataFrame(df)
        


g1
g4
g5


/Users/butis2000/Documents/Tec/Evolutionary Computation/GA/Problems.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  res = 3*ind[0] + 0.000001*ind[0]**3 + 2*ind[1] + 0.000002/(3*ind[1]**3)


g6


In [47]:
final_report['g1'].head()

,best_eval,feasible,best_eval_de,best_solution,best_solution_de,violated_constraints,feasible_de,violated_constrainsts_de
0,-138,True,-129.219661,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.41340224198642095, 0.454473190853375, 0.603...",[],True,[]
1,-138,True,-127.318660,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.301340055037242, 0.47376684018924914, 0.999...",[],False,[]
2,-138,True,-121.819975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.7432860320920208, 0.9296731391650711, 1.569...",[],False,[7]
3,-138,True,-134.530129,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.5010581477641685, 0.3832215384291808, 0.166...",[],False,[6]
4,-138,True,-124.307110,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.9353187028959575, 0.18738628024135226, 0.74...",[],False,[]


In [48]:
for problem_name,problem ,_,_ in problems:
    print('Problem: ', problem_name)
    report = generate_report(final_report[problem_name])
    print_report(report)
    print('\n\n')

Problem:  g1
Standard Deviation:

  - GA:0.0
  - Differentiial Evolution: 6.197751423861421
  - Wilcoxon Rank-Sum Test:True

Best Solution:

  - GA:
    - Score: -138.000000
    - Feasible: True

    - Solution: [0 0 0 0 0 0 0 0 0 0 0 0 0]

    - Violated Constraints: []

  - DE:
    - Score: -144.576645
    - Feasible: False

    - Solution: [-0.2701731546412683, -1.6870760288187527, 0.038313550958509544, 0.6036065382122873, 2.8273029215968544, 3.172674857450684, 2.0133443965162896, -0.08780117210212508, -1.937884781368428, 2.7280496698756025, -4.822423085619829, -4.241379018295736, -2.9776376252408454]

    - Violated Constraints: [3, 4]

Non-Feasible Solutions:

  - GA:0
  - Differentiial Evolution: 24
  - Wilcoxon Rank-Sum Test:True




Problem:  g4
Standard Deviation:

  - GA:236.5814372664572
  - Differentiial Evolution: 194.22202703133271
  - Wilcoxon Rank-Sum Test:True

Best Solution:

  - GA:
    - Score: -31964.041544
    - Feasible: True

    - Solution: [78.21155047 33.    